# Phân tích Dataset Garbage Classification
## 1. Giới thiệu tổng quan
Notebook này thực hiện các thao tác phân tích cơ bản cho dataset phân loại rác thải. Chúng ta sẽ khám phá:
- Cấu trúc thư mục
- Phân bố các lớp
- Thống kê hình ảnh
- Hiển thị mẫu dữ liệu

In [ ]:
# Import các thư viện cần thiết
import os
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np

## 2. Khám phá cấu trúc dataset
Phân tích cấu trúc thư mục và đếm số lượng mẫu trong từng lớp

In [ ]:
dataset_path = 'garbage_dataset'
classes = os.listdir(dataset_path)
class_counts = {}
image_formats = {}
image_sizes = []

for cls in classes:
    class_path = os.path.join(dataset_path, cls)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        class_counts[cls] = len(images)
        
        # Lấy thông tin định dạng ảnh
        for img_file in images[:100]:  # Kiểm tra 100 ảnh đầu tiên mỗi lớp
            img_path = os.path.join(class_path, img_file)
            try:
                with Image.open(img_path) as img:
                    fmt = img.format
                    image_formats[fmt] = image_formats.get(fmt, 0) + 1
                    image_sizes.append(img.size)
            except Exception as e:
                print(f"Lỗi khi mở ảnh {img_path}: {e}")

## 3. Phân bố các lớp
Hiển thị số lượng mẫu trong từng lớp dạng biểu đồ

In [ ]:
# Vẽ biểu đồ phân phối lớp
plt.figure(figsize=(12,6))
bars = plt.bar(class_counts.keys(), class_counts.values())
plt.title('Phân bố số lượng ảnh theo lớp')
plt.xlabel('Loại rác')
plt.ylabel('Số lượng ảnh')
plt.xticks(rotation=45)

# Thêm số liệu lên các cột
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height}',
             ha='center', va='bottom')

plt.show()

## 4. Thống kê định dạng ảnh
Kiểm tra các định dạng file ảnh tồn tại trong dataset

In [ ]:
# Hiển thị phân bố định dạng ảnh
print("\nThống kê định dạng ảnh:")
for fmt, count in image_formats.items():
    print(f"- {fmt}: {count} ảnh")

# Thống kê kích thước ảnh
widths, heights = zip(*image_sizes)
print(f"\nKích thước ảnh trung bình: {np.mean(widths):.1f}x{np.mean(heights):.1f}")
print(f"Kích thước phổ biến nhất: {pd.Series(image_sizes).mode()[0]}")

## 5. Hiển thị mẫu ảnh
Xem trước một số ảnh đại diện từ các lớp

In [ ]:
# Hiển thị ảnh mẫu
plt.figure(figsize=(15,10))
for i, cls in enumerate(classes[:5]):  # Hiển thị 5 lớp đầu
    sample_image = os.listdir(os.path.join(dataset_path, cls))[0]
    img_path = os.path.join(dataset_path, cls, sample_image)
    
    plt.subplot(2,3,i+1)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.title(f"{cls}\n{img.size[0]}x{img.size[1]}")
    plt.axis('off')

plt.tight_layout()
plt.show()

## 6. Phân tích chi tiết
Tạo bảng thống kê tổng hợp

In [ ]:
# Tạo DataFrame chứa thông tin thống kê
stats_df = pd.DataFrame({
    'Lớp': list(class_counts.keys()),
    'Số lượng': list(class_counts.values()),
    'Tỷ lệ (%)': [count/sum(class_counts.values())*100 for count in class_counts.values()]
})

print("\nBảng thống kê tổng hợp:")
display(stats_df.style.background_gradient(cmap='Blues'))

## 7. Kết luận
Từ các phân tích trên, chúng ta có thể rút ra các nhận xét ban đầu:
- Dataset có **{total_classes}** lớp với tổng số **{total_images}** ảnh
- Lớp phổ biến nhất là **{max_class}** chiếm **{max_percent:.1f}%** dataset
- Định dạng ảnh chủ yếu là **{main_format}**
- Kích thước ảnh trung bình là **{avg_width}x{avg_height}** px

**Khuyến nghị:**
- Cân bằng dataset nếu có lớp quá chênh lệch
- Chuẩn hóa kích thước ảnh trước khi training
- Kiểm tra thêm về chất lượng ảnh và metadata